In [48]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [49]:
train = pd.read_csv('/kaggle/input/shelter-animal-outcomes/train.csv.gz')
train

In [50]:
test = pd.read_csv('/kaggle/input/shelter-animal-outcomes/test.csv.gz')
test

In [51]:
all_data = pd.concat([train, test])
all_data

In [52]:
all_data_2 = all_data.drop(columns=['AnimalID','DateTime','OutcomeType','OutcomeSubtype','ID'])
all_data_2

In [53]:
all_data['DateTime'] = pd.to_datetime(all_data['DateTime'])
all_data_2['Year'] = all_data['DateTime'].dt.year
all_data_2['Month'] = all_data['DateTime'].dt.month
all_data_2['Day'] = all_data['DateTime'].dt.day
all_data_2['Hour'] = all_data['DateTime'].dt.hour
all_data_2['Minute'] = all_data['DateTime'].dt.minute

In [54]:
all_data_2

In [55]:
all_data_2['Color'].unique()

In [31]:
all_data_2['Breed'].value_counts()

In [32]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [33]:
category = all_data_2.columns[all_data_2.dtypes==object]
category

In [34]:
for i in category:
    if i != 'AgeuponOutcome':
        all_data_2[i] = le.fit_transform(list(all_data_2[i]))
all_data_2

In [35]:
all_data_2['AgeuponOutcome'].value_counts()

In [36]:
tmp = []
for day in all_data_2['AgeuponOutcome']:
    day_split = str(day).split(' ')
    #print(day_split)
    if day_split[0] == 'nan':
        day = -1
    elif day_split[1] == 'year' or day_split[1] == 'years':
        day = 365 * int(day_split[0])
    elif day_split[1] == 'month' or day_split[1] == 'months':
        day = 31 * int(day_split[0])
    elif day_split[1] == 'week' or day_split[1] == 'weeks':
        day = 7 * int(day_split[0])
    elif day_split[1] == 'day' or day_split[1] == 'days':
        day = int(day_split[0])
    else:
        print(day)   
    tmp.append(day)
all_data_2.loc[:,'AgeuponOutcome'] = tmp

In [37]:
all_data_2

In [38]:
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [39]:
train_2

In [40]:
test_2

In [41]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(verbose=300, task_type='GPU', iterations=10000) #learning_rate=0.018638, 
cbc.fit(train_2, train['OutcomeType'], early_stopping_rounds=25) #eval_set=(x_val, y_val)

In [42]:
result = cbc.predict_proba(test_2)

In [43]:
sub = pd.read_csv('/kaggle/input/shelter-animal-outcomes/sample_submission.csv.gz')
sub

In [44]:
sub.iloc[:,1:] = result
sub

In [45]:
sub.to_csv('sub.csv',index=False)